Compute some values from Spinorama following Olive&al

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from scipy.stats import linregress
from src.spinorama.load_parse import parse_all_speakers, parse_graphs_speaker
from src.spinorama.graph import graph_params_default

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
# df = parse_graphs_speaker('Ascend Acoustics', 'Ascend Acoustics Sierra 2', 'klippel')
# df = parse_graphs_speaker('JBL', 'JBL 305P Mark ii', 'klippel')
# df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
df = parse_graphs_speaker('Ocean Way', 'Ocean Way HR5', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [ ]:
def octave(N):
    """compute 1/N octave band"""
    p = pow(2,1/N)
    p_band= pow(2,1/(2*N))
    iter = int((N*10+1)/2)
    center = [1290 / p**i for i in range(iter,0,-1)]+[1000*p**i for i in range(0,iter,1)]
    return [(c/p_band,c*p_band) for c in center]


In [ ]:
def aad(dfu):
    # mean betwenn 200hz and 400hz
    y_ref = np.mean(dfu.loc[(dfu.Freq>=200) & (dfu.Freq<=400)].dB)
    print(y_ref)
    sum = 0
    n = 0
    # 1/20 octave
    for (omin, omax) in octave(20):
        # 100hz to 16k hz
        if omin < 100:
            continue
        if omax > 16000:
            break
        selection = dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)]
        if selection.shape[0] > 0:
            sum += abs(y_ref-np.mean(selection.dB))
            n += 1
    return sum/n

def nbd(dfu):
    sum = 0
    n = 0
    # 1/2 octave
    for (omin, omax) in octave(2):
        # 100hz to 12k hz
        if omin < 100:
            continue
        if omax > 12000:
            break
        y = dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)].dB
        y_avg = np.mean(y)
        # don't sample, take all points in this octave
        sum += np.mean(np.abs(y_avg-y))
        n += 1
    return sum/n


def lfx(lw, sp):
    y_ref = np.mean(lw.loc[(lw.Freq>=300) & (lw.Freq<=10000)].dB)-6
    # find first freq such that y[freq]<y_ref-6dB
    y = math.log10(sp.loc[(sp.Freq<300)&(sp.dB<=y_ref)].Freq.max())
    return y


def lfq(lw, sp, lfx_log):
    lfx = pow(10,lfx_log)
    sum = 0
    n = 0
    for (omin, omax) in octave(2):
        # 100hz to 12k hz
        if omin < lfx:
            continue
        if omax > 300:
            break
        s_lw = lw.loc[(lw.Freq>=omin) & (lw.Freq<omax)]
        s_sp = sp.loc[(sp.Freq>=omin) & (sp.Freq<omax)]
        if s_lw.shape[0] > 0 and s_sp.shape[0] > 0:
            y_lw = np.mean(s_lw.dB)
            y_sp = np.mean(s_sp.dB)
            sum += abs(y_lw-y_sp)
            n += 1
    return sum/n

def sm(dfu):
    data = dfu.loc[(dfu.Freq>=100) & (dfu.Freq<=16000)]
    log_freq = np.log(data.Freq)
    slope, intercept, r_value, p_value, std_err = linregress(log_freq, data.dB)
    return r_value**2
                

def pref_rating(nbd_on, nbd_pir, lfx, sm_pir):
    return 12.69-2.49*nbd_on-2.99*nbd_pir-4.31*lfx+2.32*sm_pir
    
df_on  = df['CEA2034'].loc[lambda df: df.Measurements == 'On Axis']
df_lw  = df['CEA2034'].loc[lambda df: df.Measurements == 'Listening Window']
df_sp  = df['CEA2034'].loc[lambda df: df.Measurements == 'Sound Power']
df_pir = df['CEA2034'].loc[lambda df: df.Measurements == 'Sound Power']

nbd_on = nbd(df_on)
nbd_lw = nbd(df_lw)
nbd_sp = nbd(df_sp)
lfx_value = lfx(df_lw, df_sp)
sm_sp = sm(df_sp)
print(' aad={0:.2f}'.format(aad(df_on)))
print(' nbd=(onaxis={0:.2f}, lw={1:.2f}, sp={2:.2f})'.format(nbd_on, nbd_lw, nbd_sp))
print(' lfx={:.0f} hz'.format(pow(10,lfx_value)))
print(' lfq={0:.2f}'.format(lfq(df_lw, df_sp, lfx_value)))
print('  sm={0:.2f}'.format(sm_sp))
print('perf={0:.2f}'.format(pref_rating(nbd_on, nbd_sp, lfx_value, sm_sp)))

In [ ]:
from more_itertools import consecutive_groups

y_ref = np.mean(df_lw.loc[(df_lw.Freq>=300) & (df_lw.Freq<=10000)].dB)-6
lfx_range = df_sp.loc[(df_sp.Freq<300)&(df_sp.dB<=y_ref)].Freq
lfx_grouped = consecutive_groups(lfx_range.iteritems(), lambda x: x[0])
lfx_hz = list(next(lfx_grouped))[-1][1]

In [ ]:
lfx_hz